# RAG: Using Gemma2 LLM locally for question answering on private data

In this notebook, our aim is to develop a RAG system utilizing [Google's Gemma](https://ai.google.dev/gemma) model. We'll generate vectors with [Elastic's ELSER](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html) model and store them in Elasticsearch. Additionally, we'll explore semantic retrieval techniques and present the top search results as a context window to the Gemma model. Furthermore, we'll utilize the [Hugging Face transformer](https://huggingface.co/google/gemma-2b-it) library to load Gemma on a local environment.

## Setup

**Elastic Credentials** - Create an [Elastic Cloud deployment](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) to get all Elastic credentials (`ELASTIC_CLOUD_ID`,` ELASTIC_API_KEY`).

**Hugging Face Token** - To get started with the [Gemma](https://huggingface.co/google/gemma-2b-it) model, it is necessary to agree to the terms on Hugging Face and generate the [access token](https://huggingface.co/docs/hub/en/security-tokens) with `write` role.

**Gemma Model** - We're going to use [gemma-2-2b](https://huggingface.co/google/gemma-2-2b), though Google has released more models. You can use any of them i.e. [gemma-2b](https://huggingface.co/google/gemma-2b), [gemma-2b-it](https://huggingface.co/google/gemma-2b-it), [gemma-7b](https://huggingface.co/google/gemma-7b), [gemma-7b-it](https://huggingface.co/google/gemma-7b-it)

## Install packages

In [ ]:
!pip install -q -U elasticsearch langchain langchain-community transformers huggingface_hub langchain-huggingface torch

## Import packages

In [ ]:
import json
import os
from getpass import getpass
from urllib.request import urlopen

import warnings
warnings.filterwarnings('ignore')

import torch
from elasticsearch import Elasticsearch, helpers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import ElasticsearchStore
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

## Get Credentials

In [ ]:
ELASTIC_API_KEY = os.getenv("ES_API_KEY")
ELASTIC_CLOUD_ID = os.getenv("ES_CLOUD_ID")
elastic_index_name = "gemma-rag"

## Add documents

### Let's download the sample dataset and deserialize the document.

In [ ]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/datasets/workplace-documents.json"

response = urlopen(url)

workplace_docs = json.loads(response.read())

### Split Documents into Passages

In [ ]:
metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["content"])
    metadata.append(
        {
            "name": doc["name"],
            "summary": doc["summary"],
            "rolePermissions": doc["rolePermissions"],
        }
    )

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=0, separators=[" ", ",", "\n"]
)
docs = text_splitter.create_documents(content, metadatas=metadata)

## Index Documents into Elasticsearch using ELSER

Before we begin indexing, ensure you have [downloaded and deployed the ELSER model](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html#download-deploy-elser) in your deployment and is running on the ML node.

In [ ]:
es = ElasticsearchStore.from_documents(
    docs,
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name=elastic_index_name,
    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(
        model_id=".elser_model_2_linux-x86_64"
    ),
)

es

## Hugging Face login

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()

## Initialize the tokenizer with the model (`google/gemma-2-2b`)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")

## Create a `text-generation` pipeline and initialize with LLM

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer, 
    max_new_tokens=1024,    # The amount of new tokens to be generated (exclude input length)
    device=0,               # 0 for GPU, -1 for CPU
    repetition_penalty=1.1, # discourage repetitive or redundant output
    return_full_text=False, # results will not contain the original query 
)

llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0},
)

## Format Docs

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

## Create a chain using Prompt template

In [ ]:
retriever = es.as_retriever(search_kwargs={"k": 3})

template = """<bos><start_of_turn>user\nAnswer the question based only on the following context and extract out a meaningful answer. \
Please write in full sentences with correct spelling and punctuation. if it makes sense use lists. \
If the context doen't contain the answer, just respond that you are unable to find an answer. \

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Ask question

In [168]:
# Q1 - from when work from office starting?
# Q2 - What are the sales goals?
# Q3 - What are the new employee onboarding steps?
# Q4 - How to apply for vacation?
# Q5 - Can I bring my pet in the office?
# Q6 - What are the pet policy?

res = chain.invoke("How to apply for vacation?")
print(res)

 To request vacation, employees should follow these steps:

1. Check your leave balance using our online portal or by contacting HR.
2. Submit your vacation request form to your manager at least two weeks before the desired start date.
3. Your manager will review the request and provide approval or guidance if needed.
4. Once approved, confirm the details of your vacation with your manager.
